In [1]:
%pylab inline
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from vocabulary import Vocabulary
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  #交叉验证
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np
import graphviz
import pickle
import warnings

warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


## 1、特征为residue_vocabulary，其序列残基对应词汇表所做的字符编码

In [5]:
# 将序列残基对应词典转换成特征
data = pd.read_excel('/home/dldx/R-H/code/classification/ML/data/mafft_MSA_R_H.xlsx')

vocabulary = Vocabulary.get_vocabulary_from_sequences(data.Sequence.values)  #将蛋白质序列构造成词典

In [6]:
# 数据处理
input_seq_tensor = []
input_lable = []
for i in range(179):
    # 将序列残基进行字符编码
    seq_tensor = vocabulary.seq_to_tensor(data.Sequence[i])
    input_seq_tensor.append(np.array(seq_tensor))
    
input_seq_tensor = np.array(input_seq_tensor)
input_lable = np.array(data.Tier)

In [19]:
# 设置超参数  
param_dict = {"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}
for r in range(0,31):
    # 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_seq_tensor, input_lable, train_size=0.8, random_state=r)
    # 定义模型  
    clf = tree.DecisionTreeClassifier(criterion="entropy")
    # 将模型加入超参数网格搜索和交叉验证
    clf = GridSearchCV(clf, param_grid=param_dict, cv=5) #将训练/测试数据集划分5个互斥子集
    # 训练数据集
    clf = clf.fit(Xtrain, Ytrain)   
    Y_pred = clf.predict(Xtest)
    cm = confusion_matrix(Ytest,Y_pred) # 混淆矩阵
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.2672612419124244
  sens : 46.666666666666664
  spec : 76.19047619047619
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.8055555555555556
  mcc : 0.6284117406521569
  sens : 76.47058823529412
  spec : 84.21052631578947
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.43145549730400484
  sens : 40.0
  spec : 87.5
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.8055555555555556
  mcc : 0.6245760466870612
  sens : 75.0
  spec : 85.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.75
  mcc : 0.4242640687119285
  sens : 90.9090909090909
  spec : 68.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.6373730021670221
  sens : 47.368421052631575
  spec : 94.11764705882352
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.11626802793945508
  sens : 42.857142857142854
  spec : 68.18181818181817
数据集划分 random_state：7 时，
Evaluate：
  accuracy :

## 2、特征为residue_disorder，其序列残基对应disorder所做的字符编码

In [18]:
data1 = pd.read_csv('/home/dldx/R-H/code/classification/ML/data/disorder/MSA_disorder_R_H.csv')

input_feature = pickle.load(open('/home/dldx/R-H/code/classification/ML/data/disorder/all_seq_disorder_R_H.pkl','rb'))
input_lable1 = np.array(data1.Tier)

In [20]:
# 设置超参数  
param_dict = {"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}
for r in range(0,31):
    # 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_feature, input_lable1, train_size=0.8, random_state=r)
    # 定义模型  
    clf = tree.DecisionTreeClassifier(criterion="entropy")
    # 将模型加入超参数网格搜索和交叉验证
    clf = GridSearchCV(clf, param_grid=param_dict, cv=5) #将训练/测试数据集划分5个互斥子集
    # 训练数据集
    clf = clf.fit(Xtrain, Ytrain)   
    Y_pred = clf.predict(Xtest)
    cm = confusion_matrix(Ytest,Y_pred) # 混淆矩阵
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : 0.5212401932644195
  sens : 80.0
  spec : 76.19047619047619
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : 0.5907388856996755
  sens : 70.58823529411765
  spec : 84.21052631578947
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.75
  mcc : 0.5532967699231196
  sens : 70.0
  spec : 81.25
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.8611111111111112
  mcc : 0.7416840554408851
  sens : 81.25
  spec : 90.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.8333333333333334
  mcc : 0.5806324452973928
  sens : 81.81818181818183
  spec : 84.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.8055555555555556
  mcc : 0.6779076806833005
  sens : 73.68421052631578
  spec : 88.23529411764706
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.15784782115637425
  sens : 64.28571428571429
  spec : 54.54545454545454
数据集划分 random_state：7 时，
Evaluate：
  accuracy : 0.72222222222

## 3、特征为21个无规则卷曲的关键区域长度，IDR1, IDR2, IDR3, IDR4, IDR5, IDR6, IDR7, IDR8,IDR9,IDR10,IDR11,IDR12,IDR13,IDR14,IDR15,IDR16,IDR17,IDR18,IDR19,IDR20,IDR21。

In [2]:
data2_R = pickle.load(open('/www/yang/csv/align/count_C_pickle/mafft_MSA/end_count_C_R_1.pkl','rb'))
data2_H = pickle.load(open('/www/yang/csv/align/count_C_pickle/mafft_MSA/end_count_C_H_3.pkl','rb'))

In [8]:
all_21_IDR = []
for position in range(21):
    one_IDR = []
    R_IDR = [i[position] for i in data2_R]
    H_IDR = [i[position] for i in data2_H]
 
    # 转化为二维数组
    for i in range(72):
        test =[]
        test.append(R_IDR[i])
        one_IDR.append(test)
    for i in range(107):
        test =[]
        test.append(H_IDR[i])
        one_IDR.append(test)

    # 所有IDR添加到一个数组
    all_21_IDR.append(one_IDR)

In [21]:
# 设置超参数  
param_dict = {"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}

# 21个IDR区域分别进行分类
for position in range(21):
    for r in range(0,31):
        # train_test_split 按比例划分原数据集
        Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR[position],input_lable,train_size=0.8,random_state = r)
        # 定义模型
        model = tree.DecisionTreeClassifier(criterion="entropy")
        # 将模型加入超参数网格搜索和交叉验证     
        model = GridSearchCV(model, param_grid=param_dict, cv=5)
        # 训练数据集
        model = model.fit(Xtrain, Ytrain)
        # 模型评估
        predict = model.predict(Xtest)
        cm = confusion_matrix(Ytest,predict)
        tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
        n = tp + fp + tn + fn
        accuracy = (tp + tn)/n 
        mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
        sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
        spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
        print("数据集划分 random_state：" + str(r) + " 时，")
        print("Evaluate：")
        print("  accuracy : " + str(accuracy))
        print("  mcc : " + str(mcc))
        print("  sens : " + str(sens))
        print("  spec : " + str(spec)) 
        print("============================")
    print("以上为IDR__" + str(position + 1) + "时，evaluation")
    print("***********************************************************************")

数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : -0.5532833351724882
  sens : 0.0
  spec : 95.23809523809523
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.5
  mcc : -0.659231724180059
  sens : 0.0
  spec : 94.73684210526315
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：7 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：8 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：9

数据集划分 random_state：29 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：30 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
以上为IDR__2时，evaluation
***********************************************************************
数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.5408646431520382
  sens : 26.666666666666668
  spec : 95.23809523809523
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.5
  mcc : -0.06694510787521936
  sens : 17.647058823529413
  spec : 78.94736842105263
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : 0.26527905453864553
  sens : 25.0
  spec : 87.5
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.17213259316477408
  sens : 31.25
  spec : 80.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.1632993161855452
  sens : 45.45454545454545
  spec : 72.0
数据集划分 random_state：5 时，
Evaluate：


数据集划分 random_state：16 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.3992747047523452
  sens : 15.384615384615385
  spec : 95.65217391304348
数据集划分 random_state：17 时，
Evaluate：
  accuracy : 0.3611111111111111
  mcc : -0.32524997430829383
  sens : 10.0
  spec : 46.15384615384615
数据集划分 random_state：18 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : 0.641688947919748
  sens : 9.090909090909092
  spec : 100.0
数据集划分 random_state：19 时，
Evaluate：
  accuracy : 0.5
  mcc : -0.30422978640307247
  sens : 7.6923076923076925
  spec : 73.91304347826086
数据集划分 random_state：20 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.8280786712108251
  sens : 8.333333333333332
  spec : 100.0
数据集划分 random_state：21 时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : -0.7171371656006362
  sens : 0.0
  spec : 94.44444444444444
数据集划分 random_state：22 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : -0.02652790545386455
  sens : 18.181818181818183
  spec : 80.0
数据集划分 random_state：23 时，
Evaluate：

数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : 0.3951316644589048
  sens : 20.0
  spec : 93.75
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.5140603704975992
  sens : 25.0
  spec : 95.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.4526019054848144
  sens : 18.181818181818183
  spec : 96.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.7405316311773545
  sens : 26.31578947368421
  spec : 100.0
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.7928249671720918
  sens : 7.142857142857142
  spec : 100.0
数据集划分 random_state：7 时，
Evaluate：
  accuracy : 0.8055555555555556
  mcc : 0.5263613559678152
  sens : 25.0
  spec : 96.42857142857143
数据集划分 random_state：8 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.37113480951260275
  sens : 14.285714285714285
  spec : 95.45454545454545
数据集划分 random_state：9 时，
Evaluate：
  accuracy : 0.5833333333333334
  m

数据集划分 random_state：26 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：27 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：28 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：29 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：30 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
以上为IDR__7时，evaluation
***********************************************************************
数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.5555555555555556
  m

...

数据集划分 random_state：26 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：27 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : -0.06900655593423542
  sens : 12.5
  spec : 82.14285714285714
数据集划分 random_state：28 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：29 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：30 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
以上为IDR__14时，evaluation
***********************************************************************
数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.7367883976130072
  sens : 5.88235294117647
  spec : 100.0
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : 0.6963106238227914
  sens : 15.0
  s

数据集划分 random_state：17 时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：18 时，
Evaluate：
  accuracy : 0.3888888888888889
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：19 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：20 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：21 时，
Evaluate：
  accuracy : 0.5
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：22 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：23 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：24 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：25 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：26 时，
Evaluate：
  accuracy : 0.6666666666666666
  m

数据集划分 random_state：7 时，
Evaluate：
  accuracy : 0.3611111111111111
  mcc : -0.04558423058385518
  sens : 62.5
  spec : 28.57142857142857
数据集划分 random_state：8 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：9 时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：10 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：11 时，
Evaluate：
  accuracy : 0.3888888888888889
  mcc : -0.75
  sens : 0.0
  spec : 77.77777777777779
数据集划分 random_state：12 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：13 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：14 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：15 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 rand

数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.5
  mcc : 0.6859943405700353
  sens : 10.0
  spec : 100.0
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.7669649888473704
  sens : 12.5
  spec : 100.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.75
  mcc : 0.8574929257125442
  sens : 18.181818181818183
  spec : 100.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.5
  mcc : 0.6969320524371696
  sens : 5.263157894736842
  spec : 100.0
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.015316791621349673
  sens : 14.285714285714285
  spec : 86.36363636363636
数据集划分 random_state：7 时，
Evaluate：
  accuracy : 0.5
  mcc : -0.19287918745261487
  sens : 12.5
  spec : 60.71428571428571
数据集划分 random_state：8 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.7928249671720918
  sens : 7.142857142857142
  spec : 100.0
数据集划分 random_state：9 时，
Evaluate：
  accuracy : 0.5
  mcc : 0.11325777888594835
  sens : 15.789473684210526
  spec : 88.2352

数据集划分 random_state：26 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：27 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：28 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：29 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：30 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
以上为IDR__21时，evaluation
***********************************************************************


In [4]:
data2_R[65][15]

3

In [10]:
# 重新跑关于IDR16由数据更改的情况

# 设置超参数  
param_dict = {"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}

for r in range(0,31):
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR[15],input_lable,train_size=0.8,random_state = r)
    # 定义模型
    model = tree.DecisionTreeClassifier(criterion="entropy")
    # 将模型加入超参数网格搜索和交叉验证     
    model = GridSearchCV(model, param_grid=param_dict, cv=5)
    # 训练数据集
    model = model.fit(Xtrain, Ytrain)
    # 模型评估
    predict = model.predict(Xtest)
    cm = confusion_matrix(Ytest,predict)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")
print("以上为IDR__" + str(16) + "时，evaluation")

数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：7 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：8 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：9 时，
Evaluate：
  accuracy : 0.472222222222222

In [11]:
# 将21个区域的无规则卷曲长度一起作为特征
all_R_H_IDR =[]
all_R_H_IDR.extend(data2_R)
all_R_H_IDR.extend(data2_H)

# 设置超参数  
param_dict = {"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}
for r in range(0,31):
    # 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_R_H_IDR, input_lable, train_size=0.8, random_state=r)
    # 定义模型  
    clf = tree.DecisionTreeClassifier(criterion="entropy")
    # 将模型加入超参数网格搜索和交叉验证
    clf = GridSearchCV(clf, param_grid=param_dict, cv=5) #将训练/测试数据集划分5个互斥子集
    # 训练数据集
    clf = clf.fit(Xtrain, Ytrain)   
    Y_pred = clf.predict(Xtest)
    cm = confusion_matrix(Ytest,Y_pred) # 混淆矩阵
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.4008918628686366
  sens : 53.333333333333336
  spec : 80.95238095238095
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : 0.5907388856996755
  sens : 70.58823529411765
  spec : 84.21052631578947
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.5773502691896257
  sens : 40.0
  spec : 93.75
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.2639315516295432
  sens : 75.0
  spec : 55.00000000000001
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.8055555555555556
  mcc : 0.5177731682921968
  sens : 81.81818181818183
  spec : 80.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.3916495317784727
  sens : 57.89473684210527
  spec : 76.47058823529412
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.13026778945578593
  sens : 50.0
  spec : 63.63636363636363
数据集划分 random_state：7 时，
Evaluate：